In [2]:
import pandas as pd

## Load Dataset

In [3]:
# 경로(ex: data/train.csv)는 사용자 설정마다 다를 수 있음
train = pd.read_csv("data/train.csv")

print(train.shape)
train.head()

(878049, 9)


,Dates,Category,Descript,DayOfWeek,PdDistrict,Resolution,Address,X,Y
0,2015-05-13 23:53:00,WARRANTS,WARRANT ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",OAK ST / LAGUNA ST,-122.425892,37.774599
1,2015-05-13 23:53:00,OTHER OFFENSES,TRAFFIC VIOLATION ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",OAK ST / LAGUNA ST,-122.425892,37.774599
2,2015-05-13 23:33:00,OTHER OFFENSES,TRAFFIC VIOLATION ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",VANNESS AV / GREENWICH ST,-122.424363,37.800414
3,2015-05-13 23:30:00,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,Wednesday,NORTHERN,NONE,1500 Block of LOMBARD ST,-122.426995,37.800873
4,2015-05-13 23:30:00,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,Wednesday,PARK,NONE,100 Block of BRODERICK ST,-122.438738,37.771541


In [4]:
test = pd.read_csv("data/test.csv")

print(test.shape)
test.head()

(884262, 7)


,Id,Dates,DayOfWeek,PdDistrict,Address,X,Y
0,0,2015-05-10 23:59:00,Sunday,BAYVIEW,2000 Block of THOMAS AV,-122.399588,37.735051
1,1,2015-05-10 23:51:00,Sunday,BAYVIEW,3RD ST / REVERE AV,-122.391523,37.732432
2,2,2015-05-10 23:50:00,Sunday,NORTHERN,2000 Block of GOUGH ST,-122.426002,37.792212
3,3,2015-05-10 23:45:00,Sunday,INGLESIDE,4700 Block of MISSION ST,-122.437394,37.721412
4,4,2015-05-10 23:45:00,Sunday,INGLESIDE,4700 Block of MISSION ST,-122.437394,37.721412


## Train

In [5]:
feature_names = ["X", "Y"]
feature_names

['X', 'Y']

In [6]:
X_train = train[feature_names]

print(X_train.shape)
X_train.head()

(878049, 2)


,X,Y
0,-122.425892,37.774599
1,-122.425892,37.774599
2,-122.424363,37.800414
3,-122.426995,37.800873
4,-122.438738,37.771541


In [7]:
X_test = test[feature_names]

print(X_test.shape)
X_test.head()

(884262, 2)


,X,Y
0,-122.399588,37.735051
1,-122.391523,37.732432
2,-122.426002,37.792212
3,-122.437394,37.721412
4,-122.437394,37.721412


In [8]:
label_name = "Category"

y_train = train[label_name]

print(y_train.shape)
y_train.head()

(878049,)


0          WARRANTS
1    OTHER OFFENSES
2    OTHER OFFENSES
3     LARCENY/THEFT
4     LARCENY/THEFT
Name: Category, dtype: object

### Use Random Forest

In [9]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(n_jobs=-1)
model

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=-1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [10]:
from sklearn.model_selection import cross_val_score

%time score = cross_val_score(model, X_train, y_train, cv=5, scoring="neg_log_loss").mean()
score = -1.0 * score

print("Score = {0:.5f}".format(score))

Wall time: 1min 10s
Score = 6.73608


### XGBoost

In [20]:
#!pip3 install xgboost
#!pip install xgboost

# guide: https://medium.com/@rakshithvasudev/how-i-installed-xgboost-after-a-lot-of-hassels-on-my-windows-machine-c53e972e801e
# install: https://stackoverflow.com/questions/33749735/how-to-install-xgboost-package-in-python-windows-platform/39811079#39811079
# finally works : https://deparkes.co.uk/2015/02/04/anaconda-whl-install/

In [12]:
import xgboost

In [13]:
import xgboost as xgb

model = xgb.XGBClassifier(n_estimators=15, nthread=2)
model

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=15,
       n_jobs=1, nthread=2, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [14]:
from sklearn.model_selection import cross_val_score

%time score = cross_val_score(model, X_train, y_train, cv=5, scoring="neg_log_loss").mean()
score = -1.0 * score

print("Score = {0:.5f}".format(score))

Wall time: 19min 55s
Score = 2.81218


In [15]:
%time model.fit(X_train, y_train)

Wall time: 4min 50s


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=15,
       n_jobs=1, nthread=2, objective='multi:softprob', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [16]:
predictions = model.predict_proba(X_test)

print(predictions.shape)
predictions

(884262, 39)


array([[0.01356191, 0.07389766, 0.01177335, ..., 0.06713752, 0.03883071,
        0.02099236],
       [0.01389777, 0.07572772, 0.01206491, ..., 0.04861059, 0.03979234,
        0.02151223],
       [0.01195893, 0.05197413, 0.01169497, ..., 0.05387832, 0.02412864,
        0.01387195],
       ...,
       [0.01219209, 0.07100576, 0.01137064, ..., 0.07376659, 0.03120419,
        0.01771765],
       [0.01388718, 0.06997736, 0.01205572, ..., 0.04857356, 0.03976202,
        0.01990316],
       [0.01241835, 0.05127233, 0.01158165, ..., 0.06524383, 0.02139572,
        0.01560471]], dtype=float32)

## Submit

In [17]:
submission = pd.read_csv("data/sampleSubmission.csv", index_col="Id")

print(submission.shape)
submission.head()

(884262, 39)


,ARSON,ASSAULT,BAD CHECKS,BRIBERY,BURGLARY,DISORDERLY CONDUCT,DRIVING UNDER THE INFLUENCE,DRUG/NARCOTIC,DRUNKENNESS,EMBEZZLEMENT,...,SEX OFFENSES NON FORCIBLE,STOLEN PROPERTY,SUICIDE,SUSPICIOUS OCC,TREA,TRESPASS,VANDALISM,VEHICLE THEFT,WARRANTS,WEAPON LAWS
Id,,,,,,,,,,,,,,,,,,,,,
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [18]:
submission = pd.DataFrame(predictions,
                          index=submission.index,
                          columns=submission.columns)

print(submission.shape)
submission.head()

(884262, 39)


,ARSON,ASSAULT,BAD CHECKS,BRIBERY,BURGLARY,DISORDERLY CONDUCT,DRIVING UNDER THE INFLUENCE,DRUG/NARCOTIC,DRUNKENNESS,EMBEZZLEMENT,...,SEX OFFENSES NON FORCIBLE,STOLEN PROPERTY,SUICIDE,SUSPICIOUS OCC,TREA,TRESPASS,VANDALISM,VEHICLE THEFT,WARRANTS,WEAPON LAWS
Id,,,,,,,,,,,,,,,,,,,,,
0,0.013562,0.073898,0.011773,0.011869,0.034131,0.012608,0.012940,0.036077,0.012624,0.012235,...,0.011690,0.013977,0.011920,0.033705,0.011574,0.015323,0.041269,0.067138,0.038831,0.020992
1,0.013898,0.075728,0.012065,0.012163,0.034976,0.012920,0.013260,0.036970,0.012937,0.012538,...,0.011979,0.014323,0.012215,0.034539,0.011861,0.015702,0.042291,0.048611,0.039792,0.021512
2,0.011959,0.051974,0.011695,0.011447,0.064669,0.012965,0.012434,0.021471,0.013255,0.012004,...,0.011410,0.014290,0.011561,0.027400,0.011346,0.015279,0.041543,0.053878,0.024129,0.013872
3,0.012403,0.073928,0.011567,0.011662,0.031971,0.012387,0.012669,0.029338,0.012499,0.011942,...,0.011485,0.013820,0.011712,0.032231,0.011372,0.014205,0.045358,0.075044,0.026464,0.017711
4,0.012403,0.073928,0.011567,0.011662,0.031971,0.012387,0.012669,0.029338,0.012499,0.011942,...,0.011485,0.013820,0.011712,0.032231,0.011372,0.014205,0.045358,0.075044,0.026464,0.017711


In [19]:
submission.to_csv("data/baseline-script.csv")